<a href="https://colab.research.google.com/github/minju00/ml/blob/main/sample_code_for_gdsc_skku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
os.environ["KERAS_BACKEND"] = "jax"

import jax
import keras_core as keras
import tensorflow as tf

In [ ]:
train, test = tf.keras.datasets.cifar10.load_data()

train_images, train_labels = train
test_images,test_labels = test
train_images = train_images/255.0
test_images = test_images/225.0
train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)

val_images = train_images[:10000]
val_labels = train_labels[:10000]
train_images = train_images[10000:]
train_labels = train_labels[10000:]

170498071/170498071 [==============================] - 7s 0us/step


In [ ]:
batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.shuffle(5000).batch(batch_size)
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.shuffle(5000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                    │ (None, 30, 30, 64)            │       1,792 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ conv2d_1 (Conv2D)                  │ (None, 28, 28, 64)            │      36,928 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ max_pooling2d (MaxPooling2D)       │ (None, 14, 14, 64)            │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ conv2d_2 (Conv2D)                  │ (None, 12, 12, 128)           │      73,856 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ conv2d_3 (Conv2D)                  │ (None, 10, 10, 128)           │     147,584 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ global_average_pooling2d           │ (None, 128)                   │           0 │
│ (GlobalAveragePooling2D)           │                               │             │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dropout (Dropout)                  │ (None, 128)                   │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense (Dense)                      │ (None, 10)                    │       1,290 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 261,450 (1021.29 KB)

 Trainable params: 261,450 (1021.29 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)


In [ ]:
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = val_ds,
    callbacks=callbacks,
)

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - acc: 0.2263 - loss: 2.0034 - val_acc: 0.4335 - val_loss: 1.5240
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - acc: 0.4325 - loss: 1.5415 - val_acc: 0.5134 - val_loss: 1.3368
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - acc: 0.5105 - loss: 1.3541 - val_acc: 0.5833 - val_loss: 1.1860
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - acc: 0.5503 - loss: 1.2391 - val_acc: 0.6161 - val_loss: 1.0831
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - acc: 0.5885 - loss: 1.1532 - val_acc: 0.6396 - val_loss: 1.0335
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - acc: 0.6170 - loss: 1.0890 - val_acc: 0.6649 - val_loss: 0.9537
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - acc: 0.6319 - loss: 1.0346 - val_acc: 0.6432 - val_loss: 1.0202
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - acc: 0.6512 - loss: 0.9827 - val_acc: 0.6953 - val_loss: 0.8773
Epoch 9/20
1250/1250 ━━━━━━━━━━━━━━━━━

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print(f'loss : {loss}')
print(f'accuracy : {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - acc: 0.7655 - loss: 0.7004
loss : 0.7128220200538635
accuracy : 0.7601000070571899


## Custom Training

In [ ]:
import os
import numpy as np
import tensorflow as tf
import keras_core as keras

Using TensorFlow backend


In [ ]:
train, test = tf.keras.datasets.cifar10.load_data()

train_images, train_labels = train
test_images,test_labels = test
train_images = train_images/255.0
test_images = test_images/225.0
train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)

val_images = train_images[:10000]
val_labels = train_labels[:10000]
train_images = train_images[10000:]
train_labels = train_labels[10000:]

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.shuffle(1024).batch(batch_size)
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.shuffle(1024).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)
mnist_input_shape = (28,28,1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

loss_fn = keras.losses.SparseCategoricalCrossentropy()


train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply(grads, model.trainable_weights)
    train_acc_metric.update_state(y, logits)
    return loss_value

In [ ]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [ ]:
import time

epochs = 20
for epoch in range(epochs):
    print(f"Epoch : {epoch}")
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_ds):
        loss_value = train_step(x_batch_train, y_batch_train)


    train_acc = train_acc_metric.result()
    print(f"Training acc : {float(train_acc):.4f}")

    train_acc_metric.reset_state()

    for x_batch_val, y_batch_val in val_ds:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_state()
    print(f"Validation acc: {float(val_acc):.4f}")
    print(f"걸린 시간 : {time.time() - start_time:.2f}s")

Epoch : 0
Training acc : 0.2988
Validation acc: 0.4341
걸린 시간 : 44.83s
Epoch : 1
Training acc : 0.4473
Validation acc: 0.5242
걸린 시간 : 11.22s
Epoch : 2
Training acc : 0.5078
Validation acc: 0.5703
걸린 시간 : 7.49s
Epoch : 3
Training acc : 0.5519
Validation acc: 0.6171
걸린 시간 : 7.60s
Epoch : 4
Training acc : 0.5876
Validation acc: 0.6432
걸린 시간 : 9.55s
Epoch : 5
Training acc : 0.6142
Validation acc: 0.6512
걸린 시간 : 7.15s
Epoch : 6
Training acc : 0.6368
Validation acc: 0.6835
걸린 시간 : 7.57s
Epoch : 7
Training acc : 0.6560
Validation acc: 0.6876
걸린 시간 : 7.49s
Epoch : 8
Training acc : 0.6741
Validation acc: 0.7046
걸린 시간 : 6.69s
Epoch : 9
Training acc : 0.6857
Validation acc: 0.7074
걸린 시간 : 8.22s
Epoch : 10
Training acc : 0.7002
Validation acc: 0.7201
걸린 시간 : 6.65s
Epoch : 11
Training acc : 0.7129
Validation acc: 0.7297
걸린 시간 : 7.51s
Epoch : 12
Training acc : 0.7206
Validation acc: 0.7244
걸린 시간 : 7.20s
Epoch : 13
Training acc : 0.7331
Validation acc: 0.7465
걸린 시간 : 6.90s
Epoch : 14
Training acc : 0.